# 2.3.1 節

In [1]:
(define (show-lines . xs)
  (define (iter xs)
    (cond ((null? xs) #t)
          (else (display (car xs))
                (newline)
                (iter (cdr xs)))))
  (iter xs))
(define nil (list))
(define (push x xs) (append xs (list x)))
(define (filter predicate seq)
  (cond ((null? seq) nil)
        ((predicate (car seq))
         (cons (car seq) (filter predicate (cdr seq))))
        (else (filter predicate (cdr seq)))))
(define (accumulate op init seq)
  (if (null? seq) init
      (op (car seq) (accumulate op init (cdr seq)))))

In [2]:
(define (atom? a)
  (and (not (null? a))
       (not (pair? a))))

## Ex. 2.53
`'(a b)`と`(list 'a 'b)`の同義性さえ分かればあとは自然である。
（括弧そのものをquoteすることはできない。）

In [3]:
(show-lines
 (list 'a 'b 'c)
 (list (list 'george))
 (cdr '((x1 x2) (y1 y2)))
 (cadr '((x1 x2) (y1 y2)))
 (pair? (car '(a short list)))
 (memq 'red '((red shoes) (blue socks)))
 (memq 'red '(red shoes blue socks))
 )

(a b c)
((george))
((y1 y2))
(y1 y2)
False
False
(red shoes blue socks)


#t

## Ex. 2.54
深い再帰

In [4]:
(define (equal? xs ys)
  (cond ((null? xs) (null? ys))
        ((atom? xs) (eq? xs ys))
        (else (and (equal? (car xs) (car ys))
                   (equal? (cdr xs) (cdr ys))))))

(show-lines
 (equal? '(this is a list) '(this is a list))
 (equal? '(this is a list) '(this (is a) list)))

True
False


#t

## Ex. 2.55

In [5]:
(car ''abracadabra)

quote

上記入力は次の式の糖衣構文である。

``
(car (quote (quote abracadabra)))``

これは、次のように解釈される。（評価？）

``(car ((quote quote) (quote abracadabra)))``

再度糖衣構文を用いてわかりやすく書くと、

``(car ('quote 'abratadabra))``

# 2.3.2 節

In [6]:
(define (variable? x) (symbol? x))
(define (same-variable? v1 v2)
  (and (variable? v1) (variable? v2) (eq? v1 v2)))
(define (make-sum a1 a2) (list '+ a1 a2))
(define (make-product m1 m2) (list '* m1 m2))
(define (sum? x) (and (pair? x) (eq? (car x) '+)))
(define (addend s) (cadr s))
(define (augend s) (caddr s))
(define (product? x) (and (pair? x) (eq? (car x) '*)))
(define (multiplier p) (cadr p))
(define (multiplicand p) (caddr p))

In [7]:
(define (deriv exp var)
  (cond ((number? exp) 0)
        ((variable? exp) (if (same-variable? exp var) 1 0))
        ((sum? exp) (make-sum (deriv (addend exp) var)
                              (deriv (augend exp) var)))
        ((product? exp) (make-sum
                         (make-product (multiplier exp)
                                       (deriv (multiplicand exp) var))
                         (make-product (deriv (multiplier exp) var)
                                       (multiplicand exp))))
        (else (error "unknown expression type: DERIV" exp))))

In [8]:
(show-lines
 (deriv '(+ x 3) 'x)
 (deriv '(* x y) 'x)
 (deriv '(* (* x y) (+ x 3)) 'x))

(+ 1 0)
(+ (* x 0) (* 1 y))
(+ (* (* x y) (+ 1 0)) (* (+ (* x 0) (* 1 y)) (+ x 3)))


#t

In [9]:
(define (=number? exp num) (and (number? exp) (= exp num)))
(define (make-sum a1 a2)
  (cond ((=number? a1 0) a2)
        ((=number? a2 0) a1)
        ((and (number? a1) (number? a2)) (+ a1 a2))
        (else (list '+ a1 a2))))
(define (make-product m1 m2)
  (cond ((or (=number? m1 0) (=number? m2 0)) 0)
        ((=number? m1 1) m2)
        ((=number? m2 1) m1)
        ((and (number? m1) (number? m2)) (* m1 m2))
        (else (list '* m1 m2))))

In [10]:
(show-lines
 (deriv '(+ x 3) 'x)
 (deriv '(* x y) 'x)
 (deriv '(* (* x y) (+ x 3)) 'x))

1
y
(+ (* x y) (* y (+ x 3)))


#t

## Ex. 2.56

In [11]:
(define (make-exponentiation base exponent)
  (cond ((=number? base 0) 0)
        ((or (=number? base 1) (=number? exponent 0)) 1)
        ((=number? exponent 1) base)
        ((and (number? base) (number? exponent)) (exp base exponent))
        (else (list '** base exponent))))
(define (exponentiation? exp) (and (pair? exp) (eq? (car exp) '**)))
(define (base exp) (cadr exp))
(define (exponent exp) (caddr exp))

In [12]:
(define (deriv exp var)
  (cond ((number? exp) 0)
        ((variable? exp) (if (same-variable? exp var) 1 0))
        ((sum? exp) (make-sum (deriv (addend exp) var)
                              (deriv (augend exp) var)))
        ((product? exp) (make-sum
                         (make-product (multiplier exp)
                                       (deriv (multiplicand exp) var))
                         (make-product (deriv (multiplier exp) var)
                                       (multiplicand exp))))
        ((exponentiation? exp) (make-product
                                (make-product (exponent exp)
                                              (make-exponentiation
                                               (base exp)
                                               (- (exponent exp) 1)))
                                (deriv (base exp) var)))
        (else (error "unknown expression type: DERIV" exp))))

In [13]:
(deriv '(** (* x y) 2) 'x)

(* (* 2 (* x y)) y)

## Ex. 2.57
動くし、理に適っているのだが、もう少し綺麗にならないものか。

`addend`はそのまま。`augend`は第二成分以降を返せばいい。
問題は、`make-sum`において任意個数の項を適切に簡約できるかどうか。
考え方は以下のとおり。`make-product`もほぼ同様。
1. 引数を「定数」と「それ以外」に分け、「定数」は合計する。
2. 0を簡約する。
 1. 「定数」の合計が0である場合は、「それ以外」を「項」とする。
 2. 「定数」の合計が0でない場合は、「それ以外」と「定数」を合わせて「項」とする。
3. 「項」を適切な形式で表現する。
 1. 「項」が空である場合は、0を返す。
 2. 「項」がシングルトンである場合は、その唯一の要素を返す。
 3. 「項」がそれ以上である場合は、先頭に`'+`を付加して返す。

In [14]:
(define (single? xs) (and (pair? xs) (null? (cdr xs))))
(define (product xs) (accumulate * 1 xs))


(define (make-sum . as)
  (let ((const (sum (filter number? as)))
        (others (filter (lambda (a) (not (number? a))) as)))
    (let ((terms (if (= const 0) others (push const others))))
      (cond ((null? terms) 0)
            ((single? terms) (car terms))
            (else (cons '+ terms))))))
 (define (addend s) (cadr s))
(define (augend s)
  (let ((followings (cddr s)))
    (if (single? followings) (car followings)
        (cons '+ followings))))

(define (make-product . ms)
  (let ((const (product (filter number? ms)))
        (others (filter (lambda (m) (not (number? m))) ms)))
    (let ((factors (cond ((= const 0) (list 0))
                      ((= const 1) others)
                      (else (push const others)))))
      (cond ((null? factors) 1)
            ((single? factors) (car factors))
            (else (cons '* factors))))))
 (define (multiplier p) (cadr p))
(define (multiplicand p)
  (let ((followings (cddr p)))
    (if (single? followings) (car followings)
        (cons '* followings))))

In [15]:
(deriv '(* x y (+ x 3)) 'x)

(+ (* x y) (* y (+ x 3)))

## Ex. 2.58

### a.


In [20]:
(define (make-sum a1 a2)
  (cond ((=number? a1 0) a2)
        ((=number? a2 0) a1)
        ((and (number? a1) (number? a2)) (+ a1 a2))
        (else (list a1 '+ a2))))
(define (make-product m1 m2)
  (cond ((or (=number? m1 0) (=number? m2 0)) 0)
        ((=number? m1 1) m2)
        ((=number? m2 1) m1)
        ((and (number? m1) (number? m2)) (* m1 m2))
        (else (list m1 '* m2))))
(define (sum? x) (and (pair? x) (pair? (cdr x)) (eq? (cadr x) '+)))
(define (addend s) (car s))
;(define (augend s) (caddr s)) ; no changes
(define (product? x) (and (pair? x) (pair? (cdr x)) (eq? (cadr x) '*)))
(define (multiplier p) (car p))
;(define (multiplicand p) (caddr p)) ; no changes

In [24]:
(deriv '(x + ((2 * x) * y)) 'x)

(1 + (2 * y))

### b.
ちゃんとやるにはパーサーが必要。

コンストラクタは弄らない。

In [40]:
(define (take-before x xs)
  (if (or (null? xs) (eq? (car xs) x)) nil
        (cons (car xs) (take-before x (cdr xs)))))
(define (take-after x xs)
  (cond ((null? xs) nil)
        ((eq? (car xs) x) (cdr xs))
        (else (take-after x (cdr xs)))))
(define (unlist xs) (if (single? xs) (car xs) xs))

(define (sum? x) (memq '+ x))
(define (addend s) (unlist (take-before '+ s)))
(define (augend s) (unlist (take-after '+ s)))
(define (product? x) (and (not (sum? x)) (memq '* x)))
(define (multiplier p) (unlist (take-before '* p)))
(define (multiplicand p) (unlist (take-after '* p)))

In [41]:
(deriv '(x + 2 * x * y) 'x)

(1 + (2 * y))

# 2.3.3 節

In [43]:
(define (element-of-set? x set)
  (cond ((null? set) #f)
        ((equal? x (car set)) #t)
        (else (element-of-set? x (cdr set)))))
(define (adjoint-set x set)
  (if (element-of-set? x set)
      set
      (cons x set)))
(define (intersection-set set1 set2)
  (cond ((or (null? set1) (null? set2)) nil)
        ((element-of-set? (car set1) set2) (cons (car set1) (intersection-set (cdr set1) set2)))
        (else (intersection-set (cdr set1) set2))))

## Ex. 2.59

In [44]:
(define (union-set set1 set2)
  (if (null? set1)
      set2
      (union-set (cdr set1) (adjoint-set (car set1) set2))))

(union-set '(a b c d) '(c d e))

(b a c d e)

## Ex. 2.60

In [46]:
(define (element-of-set? x set)
  (cond ((null? set) #f)
        ((equal? x (car set)) #t)
        (else element-of-set? x (cdr set))))
(define (adjoint-set x set) (cons x set))
(define (union-set set1 set2) (append set1 set2))
;(define (intersection-set set1 set2) ...) ; no change

#f

`adjoint-set`と`union-set`のオーダーが下がる（それぞれ$\Theta(1)$と$\Theta(n)$）一方で、リストの長さが増えやすく、
そのため`element-of-set?`と`intersection-set`の効率が悪くなる傾向にある。
また、空間計算量そのものも無視できない。
要素の追加が頻繁にあって、検索が少ない応用には向いているが、
それでもときどき重複を除去するのがよいだろう。

### 補註
重複を除去するための手続き。$\Theta(n^2)$

In [51]:
(define (remove-all x ys)
  (filter (lambda (y) (not (equal? x y))) ys))
(define (unique-list set)
  (if (null? set) nil
      (cons (car set) (unique-list (remove-all (car set) (cdr set))))))

(unique-set '(2 3 2 1 3 2 2))

(2 3 1)

---

In [49]:
(define (element-of-set? x set)
  (cond ((null? set) #f)
        ((= x (car set)) #t)
        ((< x (car set)) #f)
        (else (element-of-set? x (cdr set)))))
(define (intersection-set set1 set2)
  (if (or (null? set1) (null? set2)) nil
      (let ((x1 (car set1)) (x2 (car set2)))
        (cond ((= x1 x2) (cons x1 (intersection-set (cdr set1) (cdr set2))))
              ((< x1 x2) (intersection-set (cdr set1) set2))
              ((> x1 x2) (intersection-set set1 (cdr set2)))))))

## Ex. 2.61

In [52]:
(define (adjoint-set x set)
  (cond ((null? set) (list x))
        ((= x (car set)) set)
        ((< x (car set)) (cons x set))
        (else (cons (car set) (adjoint-set x (cdr set))))))

(adjoint-set 3 '(1 2 4 5))

(1 2 3 4 5)

## Ex. 2.62

In [55]:
(define (union-set set1 set2)
  (cond ((null? set1) set2)
        ((null? set2) set1)
        (else (let ((x1 (car set1)) (x2 (car set2)))
                (cond ((= x1 x2) (cons x1 (union-set (cdr set1) (cdr set2))))
                      ((< x1 x2) (cons x1 (union-set (cdr set1) set2)))
                      ((> x1 x2) (cons x2 (union-set set1 (cdr set2)))))))))

(union-set '(1 3 5 7 9) '(2 3 5 7 11))

(1 2 3 5 7 9 11)

---

In [58]:
(define (entry tree) (car tree))
(define (left-branch tree) (cadr tree))
(define (right-branch tree) (caddr tree))
(define (make-tree entry left right)
  (list entry left right))
(define (element-of-set? x set)
  (cond ((null? set) #f)
        ((= x (entry set)) #t)
        ((< x (entry set)) (element-of-set? x (left-branch set)))
        ((> x (entry set)) (element-of-set? x (right-branch set)))))
(define (adjoint-set x set)
  (cond ((null? set) (make-tree x nil nil))
        ((= x (entry set)) set)
        ((< x (entry set)) (make-tree (entry x) (adjoint-set x (left-branch set)) (right-branch set)))
        ((> x (entry set)) (make-tree (entry x) (left-branch set) (adjoint-set x (right-branch set))))))

## Ex. 2.63

In [60]:
(define (tree->list-1 tree)
  (if (null? tree) nil
      (append (tree->list-1 (left-branch tree))
              (cons (entry tree) (tree->list-1 (right-branch tree))))))
(define (tree->list-2 tree)
  (define (copy-to-list tree result-list)
    (if (null? tree) result-list
        (copy-to-list (left-branch tree) (cons (entry tree) (copy-to-list (right-branch tree) result-list)))))
  (copy-to-list tree nil))

### a.

In [67]:
(define t1 '(7
             (3(1 () ()) (5 () ()))
             (9 () (11 () ()))))
(define t2 '(3
             (1 () ())
             (7 (5 () ()) (9 () (11 () ())))))
(define t3 '(5
             (3 (1 () ()) ())
             (9 (7 () ()) (11 () ()))))

(show-lines
 (tree->list-1 t1)
 (tree->list-2 t1)
 (tree->list-1 t2)
 (tree->list-2 t2)
 (tree->list-1 t3)
 (tree->list-2 t3))

(1 3 5 7 9 11)
(1 3 5 7 9 11)
(1 3 5 7 9 11)
(1 3 5 7 9 11)
(1 3 5 7 9 11)
(1 3 5 7 9 11)


#t

## Ex. 2.64

In [70]:
(define (list->tree elements)
  (car (partial-tree elements (length elements))))
(define (partial-tree elts n)
  (if (= n 0) (cons nil elts)
      (let ((left-size (quotient (- n 1) 2)))
        (let ((left-result (partial-tree elts left-size)))
          (let ((left-tree (car left-result))
                (non-left-elts (cdr left-result))
                (right-size (- n (+ left-size 1))))
            (let ((this-entry (car non-left-elts))
                  (right-result (partial-tree (cdr non-left-elts) right-size)))
              (let ((right-tree (car right-result))
                    (remaining-elts (cdr right-result)))
                (cons (make-tree this-entry
                                 left-tree
                                 right-tree)
                      remaining-elts))))))))

(list->tree '(1 3 5 7 9 11))

(5 (1 () (3 () ())) (9 (7 () ()) (11 () ())))

### 補註

In [71]:
(define (partial-tree elts n)
  (if (= n 0) (cons nil elts)
      (let* ((left-size (quotient (- n 1) 2))
             (left-result (partial-tree elts left-size))
             (left-tree (car left-result))
             (non-left-elts (cdr left-result))
             (right-size (- n (+ left-size 1)))
             (this-entry (car non-left-elts))
             (right-result (partial-tree (cdr non-left-elts) right-size))
             (right-tree (car right-result))
             (remaining-elts (cdr right-result)))
        (cons (make-tree this-entry
                         left-tree
                         right-tree)
              remaining-elts))))

(list->tree '(1 3 5 7 9 11))

(5 (1 () (3 () ())) (9 (7 () ()) (11 () ())))

## Ex. 2.65

In [73]:
(define (union-set-as-list set1 set2)
  (cond ((null? set1) set2)
        ((null? set2) set1)
        (else (let ((x1 (car set1)) (x2 (car set2)))
                (cond ((= x1 x2) (cons x1 (union-set-as-list (cdr set1) (cdr set2))))
                      ((< x1 x2) (cons x1 (union-set-as-list (cdr set1) set2)))
                      ((> x1 x2) (cons x2 (union-set-as-list set1 (cdr set2)))))))))
(define (intersection-set-as-list set1 set2)
  (if (or (null? set1) (null? set2)) nil
      (let ((x1 (car set1)) (x2 (car set2)))
        (cond ((= x1 x2) (cons x1 (intersection-set-as-list (cdr set1) (cdr set2))))
              ((< x1 x2) (intersection-set-as-list (cdr set1) set2))
              ((> x1 x2) (intersection-set-as-list set1 (cdr set2)))))))
(define (union-set set1 set2)
  (list->set (union-set-as-list (tree->list1 set1) (tree->list1 set2))))
(define (intersection-set set1 set2)
  (list->set (intersection-set-as-list (tree->list1 set1) (tree->list set2))))